In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import os
import seaborn as sns
from datetime import datetime, timedelta

In [2]:
animaldf = pd.read_csv("animal_info.csv", sep = ";")
animaldf.columns = [col.replace(" ", "_").lower().replace("-", "") for col in animaldf.columns]
fresh_dates = animaldf[["animal_number", "last_calving_date"]]
fresh_dates["fresh_date"] = pd.to_datetime(fresh_dates["last_calving_date"], format = '%m/%d/%Y')
fresh_dates = fresh_dates[~fresh_dates.fresh_date.isnull()]
fresh_dates = fresh_dates.drop("last_calving_date", axis = 1)
fresh_dates = fresh_dates[(~fresh_dates.animal_number.isna())]

df = pd.read_csv(r"more_reports\oneyear_data.csv", sep = ";")
df.columns = [col.replace(" ", "_").lower().replace("-", "") for col in df.columns]
df["date"] = pd.to_datetime(df["date"], format = '%m/%d/%Y')
merged_df =  pd.merge(fresh_dates, df,  left_on="animal_number", right_on = "animal_number")

merged_df["dim"] = (merged_df["date"] - merged_df["fresh_date"]).dt.days
df2 = merged_df[merged_df.dim >= 0]


C:\Users\marie\AppData\Local\Temp\ipykernel_7244\2556778765.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fresh_dates["fresh_date"] = pd.to_datetime(fresh_dates["last_calving_date"], format = '%m/%d/%Y')


# Average milk yield

## averge daily milk yield current lactation

In [3]:
yield_df = df2[['animal_number', 'date', 'yield', 'yield_lr', 'yield_lf', 'yield_rr', 'yield_rf', 'dim']]
dyield_df = yield_df.groupby(['animal_number', 'date', 'dim']).sum(['yield', 'yield_lr', 'yield_lf', 'yield_rr', 'yield_rf']).reset_index()
dyield_gdf = dyield_df.groupby('animal_number').agg({'yield': 'sum', 'yield_lr':'sum', 'yield_lf': 'sum', 'yield_rr':'sum', 'yield_rf':'sum', 'dim':'max'}).reset_index()
dyield_gdf = dyield_gdf.rename(columns = {'yield':'dyield_cl', 'yield_lr':'dyield_lr_cl', 'yield_lf': 'dyield_lf_cl', 'yield_rr': 'dyield_rr_cl', 'yield_rf': 'dyield_rf_cl'})

dyield_gdf["dyield_dim_cl"] = dyield_gdf["dyield_cl"]/ dyield_gdf['dim']
dyield_gdf["dyield_rr_dim_cl"] = dyield_gdf["dyield_rr_cl"]/ dyield_gdf['dim']
dyield_gdf["dyield_rf_dim_cl"] = dyield_gdf["dyield_rf_cl"]/ dyield_gdf['dim']
dyield_gdf["dyield_lr_dim_cl"] = dyield_gdf["dyield_lr_cl"]/ dyield_gdf['dim']
dyield_gdf["dyield_lf_dim_cl"] = dyield_gdf["dyield_lf_cl"]/ dyield_gdf['dim']
yield1 = dyield_gdf.copy()
yield1

,animal_number,dyield_cl,dyield_lr_cl,dyield_lf_cl,dyield_rr_cl,dyield_rf_cl,dim,dyield_dim_cl,dyield_rr_dim_cl,dyield_rf_dim_cl,dyield_lr_dim_cl,dyield_lf_dim_cl
0,254,13588.28,5247.39,0.00,4539.14,3801.75,320,42.463375,14.184813,11.880469,16.398094,0.000000
1,453,12669.26,3336.17,2897.72,3279.19,3156.18,291,43.536976,11.268694,10.845979,11.464502,9.957801
2,454,11622.23,3276.69,2432.15,3782.15,2131.24,241,48.225021,15.693568,8.843320,13.596224,10.091909
3,497,13081.89,3717.33,2832.37,3594.37,2937.82,309,42.336214,11.632265,9.507508,12.030194,9.166246
4,571,1102.89,392.54,225.47,309.63,175.25,28,39.388929,11.058214,6.258929,14.019286,8.052500
...,...,...,...,...,...,...,...,...,...,...,...,...
301,1501,556.69,146.79,134.60,132.51,142.79,21,26.509048,6.310000,6.799524,6.990000,6.409524
302,1503,162.35,38.09,41.25,37.92,45.09,7,23.192857,5.417143,6.441429,5.441429,5.892857
303,1504,117.47,21.56,29.17,27.01,39.73,12,9.789167,2.250833,3.310833,1.796667,2.430833
304,1507,78.02,1.03,9.17,35.07,32.75,7,11.145714,5.010000,4.678571,0.147143,1.310000


## averge daily milk yield last 7 days

In [4]:
ddf = df2[(df2.date >= '2023-11-10') & (df2.date < '2023-11-17') & (df2.dim >= 7)]
yield_df = ddf[['animal_number', 'date', 'yield', 'yield_lr', 'yield_lf', 'yield_rr', 'yield_rf','dim']]
dyield_df = yield_df.groupby(['animal_number', 'date', 'dim']).sum('yield').reset_index()
dyield_gdf = dyield_df.groupby('animal_number').agg({'yield': 'sum', 'yield_lr':'sum', 'yield_lf': 'sum', 'yield_rr':'sum', 'yield_rf':'sum'}).reset_index()
dyield_gdf = dyield_gdf.rename(columns = {'yield':'dyield_7d', 'yield_lr':'dyield_lr_7d', 'yield_lf': 'dyield_lf_7d', 'yield_rr': 'dyield_rr_7d', 'yield_rf': 'dyield_rf_7d'})

dyield_gdf["adyield_nd_7d"] = dyield_gdf["dyield_7d"]/7
dyield_gdf["adyield_rr_nd_7d"] = dyield_gdf["dyield_rr_7d"]/ 7
dyield_gdf["adyield_rf_nd_7d"] = dyield_gdf["dyield_rf_7d"]/ 7
dyield_gdf["adyield_lr_nd_7d"] = dyield_gdf["dyield_lr_7d"]/ 7
dyield_gdf["adyield_lf_nd_7d"] = dyield_gdf["dyield_lf_7d"]/ 7
yield2 = dyield_gdf.copy()

## average visit yield current lactation 

In [5]:
yield_df = df2[['animal_number', 'yield', 'yield_lr', 'yield_lf', 'yield_rr', 'yield_rf', 'dim']]
vyield_df = yield_df.groupby('animal_number').agg({'yield':'mean', 'yield_lr':'mean', 'yield_lf': 'mean', 'yield_rf': 'mean', 'yield_rr': 'mean'}).reset_index()
vyield_df = vyield_df.rename(columns = {'yield':'vyield_cl', 'yield_lr':'vyield_lr_cl', 'yield_lf': 'vyield_lf_cl', 'yield_rr': 'vyield_rr_cl', 'yield_rf': 'vyield_rf_cl'})
yield3 = vyield_df.copy()

## average visit yield last 7 days

In [6]:
ddf = df2[(df2.date >= '2023-11-10') & (df2.date < '2023-11-17') & (df2.dim >= 7)]
yield_df = ddf[['animal_number', 'yield', 'yield_lr', 'yield_lf', 'yield_rr', 'yield_rf', 'dim']]
vyield_df = yield_df.groupby('animal_number').agg({'yield':'mean', 'yield_lr':'mean', 'yield_lf': 'mean', 'yield_rf': 'mean', 'yield_rr': 'mean'}).reset_index()
vyield_df = vyield_df.rename(columns = {'yield':'vyield_7d', 'yield_lr':'vyield_lr_7d', 'yield_lf': 'vyield_lf_7d', 'yield_rr': 'vyield_rr_7d', 'yield_rf': 'vyield_rf_7d'})
yield4 = vyield_df.copy()

# Merge and save

In [7]:
integrated_df = yield1.merge(yield2, how='left', on = ["animal_number"]).merge(yield3, how='left', on = ["animal_number"]).merge(yield4, how='left', on = ["animal_number"])
integrated_df.columns = [col.replace("rr", "rb").replace("lr", "lb").replace("animal_number", "cow") for col in integrated_df.columns]
integrated_df.to_csv(r"data_out\yield_integrated.csv", index = False)